In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Configuring Retries in the Gen AI SDK

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/sdk/retries/configure_retries.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fsdk%2Fretries%2Fconfigure_retries.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/sdk/retries/configure_retries.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/sdk/retries/configure_retries.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<p>
<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/sdk/retries/configure_retries.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/sdk/retries/configure_retries.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/sdk/retries/configure_retries.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/sdk/retries/configure_retries.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/sdk/retries/configure_retries.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>
</p>

| Author |
| --- |
| [Eric Dong](https://github.com/gericdong) |

## Overview

When interacting with Gemini through Gemini API, transient network errors or service disruptions can occur. Implementing a retry strategy is crucial for building robust and reliable applications. This guide explains how to configure HTTP retry options for the [Google Gen AI SDK](https://docs.cloud.google.com/vertex-ai/generative-ai/docs/sdks/overview).


### **Why Configure Retries?**

By default, the Gen AI SDK has some level of retry for certain types of errors. However, for fine-grained control over the retry behavior, you can configure your own retry strategy. This is particularly important for:

- **Handling transient errors**:  Automatically retry requests that fail due to temporary issues, such as network timeouts or temporary server unavailability (e.g., 5xx errors).
- **Managing rate limits**: If you anticipate hitting rate limits (e.g., 429 errors), a retry strategy with exponential backoff can help manage the request rate and avoid overwhelming the service.
- **Improving application resilience**: Make your application more resilient to intermittent issues, leading to a better user experience.


## Getting Started

### Install Google Gen AI SDK for Python


In [ ]:
%pip install --upgrade --quiet google-genai

### Import libraries


In [ ]:
import os
import sys

from IPython.display import Markdown, display
from google import genai
from google.genai import types

### Authenticate your notebook environment

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Authenticate your Google Cloud Project for Vertex AI

You can use a Google Cloud Project or an API Key for authentication. This tutorial uses a Google Cloud Project.

- [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)

In [ ]:
# fmt: off
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "global"

## **Option 1**: Configuring Retries at the Client Level

The Gen AI SDK provides the `genai.types.HttpRetryOptions` class to configure a custom retry policy. You can then apply this policy to the `genai.Client` using `genai.types.HttpOptions`.

### **Step 1**: Define `HttpRetryOptions`

First, create an instance of `HttpRetryOptions` and specify the desired retry behavior. Here are some of the key parameters:

- `initial_delay`: The initial delay in seconds before the first retry.
- `attempts`: The maximum number of retry attempts.
- `exp_base`: The base for the exponential backoff calculation.
- `max_delay`: The maximum delay in seconds between retries.
- `jitter`: A factor to add a random delay to the backoff, which can help avoid a "thundering herd" problem.
- `http_status_codes`: A list of HTTP status codes that should trigger a retry.

In [ ]:
# Configure the retry policy.
retry_options = types.HttpRetryOptions(
    initial_delay=1.0,  # Initial delay of 1 second.
    attempts=5,  # Retry up to 5 times.
    exp_base=2,  # Double the delay for each subsequent retry.
    http_status_codes=[429, 500, 502, 503, 504],  # Retry on these server errors.
)

### **Step 2**: Apply to the Client with `HttpOptions`

Next, create an instance of `HttpOptions` and pass your `retry_options` to it.

You can also configure other HTTP options, such as `timeout` - Timeout for the request in milliseconds.

In [ ]:
# Configure HTTP options with the retry settings.
http_options = types.HttpOptions(
    retry_options=retry_options,
    timeout=120 * 1000,  # Set a 120-second timeout for requests.
)

### **Step 3**: Initialize the `genai.Client`

Finally, initialize the `genai.Client` with the `http_options`.

In [ ]:
# Initialize the client with the configured HTTP options.
client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION,
    http_options=http_options,
)

Now, any API calls made with this client will use the configured retry policy.

### Complete Code Example

Here is the complete code sample.

In [ ]:
client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION,
    http_options=types.HttpOptions(
        retry_options=types.HttpRetryOptions(
            initial_delay=1.0,
            attempts=10,
            http_status_codes=[429, 500, 502, 503, 504],
        ),
        timeout=120 * 1000,
    ),
)

# You can now use this client to make API calls with the custom retry policy.
# For example:
response = client.models.generate_content(
    model="gemini-3-flash-preview", contents="Tell me a story"
)

display(Markdown(response.text))

## **Option 2**: Configuring Retries at the Request Level

You can also set `HttpRetryOptions` per-request using the `config` parameter.

In [ ]:
response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents="Tell me a joke about a rabbit.",
    config=types.GenerateContentConfig(
        http_options=types.HttpOptions(
            retry_options=types.HttpRetryOptions(
               initial_delay=1.0,
               attempts=10,
               http_status_codes=[429, 500, 502, 503, 504],
            ),
            timeout=120 * 1000,
        )
    )
)

print(response.text)